In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<img src="https://www.clinicone.com.np/wp-content/uploads/2020/04/covid-vaccine-microscope.-Coronavirus-2019-nCoV-COVID-shut.jpg"/>

* Covid-19, the pandemic that no one expected to have a huge impact on the mass population. During early days people began to ingore until it became the pandemic and took a loss of human live. Since the time of its spread many countries tried different strategy to find the vaccination againt it. Now, fortunately we have landed at the time where we have got the vaccination for COVID-19, its just the beginning of the distribution of vaccinations, many countries still have not received the vaccination which we can visualize later on the graph. Anyway the ray of hope against the COVID-19 has begun and all thanks to the countries who put their soul and effort for the development of such a vaccination against COVID-19. I would like to thank personally Mr. Gabriel Preda who have made available this dataset. I am kind of new here so I will try my best to explore and visulaize the dataset.



In [ ]:
# Importing all the necessary pacakges

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('../input/covid-world-vaccination-progress/country_vaccinations.csv')

In [ ]:
df.head(4)

In [ ]:
## Lets see if there are any null values present
df.isna().sum().any()

In [ ]:
df.isna().sum()

In [ ]:
# We can clearly see that many values in the columns are missing we will fix it later

In [ ]:
df.describe(include='all').T.sort_values(by='unique')

>> From this data we can highlight some of the points:
    * Pfizer/BioNTech is the most used vaccines
    * Most of the data source about the vaccines came from Ministry OF Health.

In [ ]:
# lets make a copy of our original dataset so that we can work it on later on
df1 = df.copy()

In [ ]:
df1.head(2)

In [ ]:
# Let's group the data so that we can visualize the data more clearly
vaccine = df1.groupby(['country','vaccines','iso_code'])['total_vaccinations','people_vaccinated',
                                                                        'people_fully_vaccinated','total_vaccinations_per_hundred',
                                                                        'people_vaccinated_per_hundred'
                                                                       ].max().reset_index()

In [ ]:
vaccine.head()

In [ ]:
# Here Red color indicates the maximum number of data entries
vaccine.style.background_gradient(cmap='Reds')

In [ ]:
# lets see which country used which vaccines to fight against COVID-19
vaccines_list = list(vaccine['vaccines'].unique())

In [ ]:
for i in vaccines_list:
    country = tuple(vaccine[vaccine['vaccines']==i]['country'])
    print(f"Name of the country:{country}\n\n Used vaccines:{i}")
    print(' _ '*40)
    print(' _ '*40)

In [ ]:
vaccines_used = vaccine['vaccines'].value_counts().reset_index()
vaccines_used.columns = ['Name of Vaccines','Number of individual country']
vaccines_used

### From this data it is clear that, Pfizer/BioNTech is the vaccine used by most of the countries that is used by 33 countries till now.

In [ ]:
# Let's visualize this more clearly
fig = px.bar(vaccines_used,x='Name of Vaccines',y='Number of individual country',color='Name of Vaccines',
             height=600,width=1500
            
            )
fig.show()

## lets see which country using which vaccines in figure and this can be visualized easily with tree map and sunburst

In [ ]:
fig = px.treemap(vaccine,names='country',values='people_vaccinated',path=['vaccines','country'],
                 hover_data=['iso_code']
                )
fig.show()

In [ ]:
# using sunbrust
fig = px.sunburst(vaccine,names='country',values='people_vaccinated',path=['vaccines','country'],
                  width=1000,height=700,title='Name of vaccines per Country'
                 )
fig.show()

In [ ]:
vaccine.head(2)


## The vaccines type used in each Region

In [ ]:
fig = px.choropleth(vaccine,locations='iso_code',color='vaccines',projection='natural earth',
                    hover_name='country',height=None
                   )
fig.show()

## Lets visualize the graph region wise to see the number of people vaccinated and total vaccinations given in each region till now with the country that has been registered officially.

In [ ]:
# Number of people vaccinated till now
fig = px.choropleth(vaccine,locations='iso_code',color='people_vaccinated',projection='natural earth',
                    hover_name='country',height=None,range_color=[0,6000000],
                   )
fig.show()

In [ ]:
# Number of total vaccination used in each region
fig = px.choropleth(vaccine,locations='iso_code',color='total_vaccinations',projection='natural earth',
                    hover_name='country',height=None,range_color=[0,6000000],
                   )
fig.show()

## Now let's see some animation of the region along with date for that I have to use the original data which I copied earlier since the date is in object format I have to parse it.

In [ ]:
df2 = df.copy()

In [ ]:
df2.head(2)

In [ ]:
df2.isna().sum()

In [ ]:
## It seems that many of the iso_code of countries are missing lets see it
df2[df2['iso_code'].isnull()]['country']

In [ ]:
df2[df2['iso_code'].isnull()]['country'].count()

In [ ]:
df2[df2['iso_code'].isnull()]['country'].value_counts()

## It is clear that region of England are missing iso_code
* From the offical source https://www.iban.com/country-codes, I came to know that the iso code (Alpha Code-3) of England is GBR and also Wales, Scotland and England are the same part of the UK we will replace it with 'GBR',
 

In [ ]:
df2.iso_code.fillna('GBR', inplace=True)

In [ ]:
df2.isna().sum()

In [ ]:

df2.date = pd.to_datetime(df2['date'])

In [ ]:
df2 = df2.sort_values('date', ascending=True)

In [ ]:
df2.date = df2['date'].dt.strftime('%m/%d/%Y')

In [ ]:
df2.head(2)

In [ ]:
# timeframe animation of people vaccinated region wise
fig = px.choropleth(df2,locations='iso_code',hover_name='country',color='people_vaccinated',animation_frame='date',
                    projection='natural earth',range_color=[0,6000000],
                    color_continuous_scale='Reds'
                   ) 
fig.show()

In [ ]:
# timeframe animatiob of total_vaccinations used region wise
fig = px.choropleth(df2,locations='iso_code',hover_name='country',color='total_vaccinations',animation_frame='date',
                    projection='natural earth',range_color=[0,6000000],
                    
                    color_continuous_scale='Reds'
                   ) 
fig.show()

In [ ]:
fig = go.Figure(data=[
    go.Bar(
        name='Total Vaccinations',
        x=df2['date'],
        y=df2['total_vaccinations'],
        marker_color = 'crimson'
    
    ),
    go.Bar(
        name='People Vaccinated',
        x=df2['date'],
        y=df2['people_vaccinated'],
        marker_color = 'green'
    
    ),
    
])
fig.update_layout(
    title="Total vaccinations vs people vaccinated",
    xaxis_title = 'Date',
    yaxis_title = 'Number of count',
    barmode='stack',
    hovermode='x'

)
fig.show()

## Thankyou